In [ ]:
## Vocabularios y métricas

# Calcular vocabularios
vocab_crudo = set(tokens_crudo)
vocab_norm = set(tokens_norm)
vocab_spacy = set(tokens_spacy)

print(f"Vocabulario crudo: {len(vocab_crudo)} tokens únicos")
print(f"Vocabulario normalizado: {len(vocab_norm)} tokens únicos")
print(f"Vocabulario spaCy: {len(vocab_spacy)} tokens únicos")

# Ejemplos de tokens
print(f"\nEjemplos crudo: {list(vocab_crudo)[:10]}")
print(f"Ejemplos normalizado: {list(vocab_norm)[:10]}")
print(f"Ejemplos spaCy: {list(vocab_spacy)[:10]}")

In [ ]:
# Pipeline (c) spaCy - más lento
print("Procesando con spaCy...")
tokens_spacy = []
for i, texto in enumerate(textos):
    if i % 1000 == 0:
        print(f"Procesadas: {i}")
    tokens_spacy.extend(pipeline_spacy(texto))

print(f"Pipeline spaCy: {len(tokens_spacy)} tokens")

In [ ]:
## Procesamiento de textos

# Procesar con los 3 pipelines
print("Procesando 5,000 oraciones...")

# Pipeline (a) crudo
tokens_crudo = []
for texto in textos:
    tokens_crudo.extend(pipeline_crudo(texto))

# Pipeline (b) normalizado
tokens_norm = []
for texto in textos:
    tokens_norm.extend(pipeline_normalizado(texto))

print(f"Pipeline crudo: {len(tokens_crudo)} tokens")
print(f"Pipeline normalizado: {len(tokens_norm)} tokens")

# Proyecto 4: Pipeline de preprocesamiento

## Objetivo
Crear 3 pipelines para 5,000 oraciones:
- (a) crudo
- (b) normalizado (minúsculas, sin puntuación)
- (c) normalizado+lematizado (spaCy ES)

## Historias de usuario
- Como analista de texto, quiero comparar pipelines para reducir OOV sin perder información
- Como investigador, necesito métricas de vocabulario para optimizar preprocesamiento

## Setup reproducible

In [1]:
# Seeds para reproducibilidad
import random
import numpy as np
random.seed(42)
np.random.seed(42)

# Imports básicos
import pandas as pd
import matplotlib.pyplot as plt
import spacy
import re
from collections import Counter

print("Setup completo")

Setup completo


## Carga de datos

In [3]:
# Cargar dataset y tomar 5,000 oraciones
df = pd.read_csv('dataset/nlp_prueba_cc0c2_large.csv')
textos = df['Texto'].head(5000).tolist()

print(f"Oraciones cargadas: {len(textos)}")
print(f"Ejemplo: {textos[0]}")

Oraciones cargadas: 5000
Ejemplo: La tokenización es clave para procesar texto


## Pipelines

In [4]:
# (a) Pipeline crudo
def pipeline_crudo(texto):
    """Tokenización básica por espacios"""
    return texto.split()

# (b) Pipeline normalizado
def pipeline_normalizado(texto):
    """Minúsculas + sin puntuación"""
    # minúsculas
    texto = texto.lower()
    # quitar puntuación
    texto = re.sub(r'[.,;:!?()\[\]{}"\']', '', texto)
    return texto.split()

print("Pipelines definidos")

Pipelines definidos


In [5]:
# (c) Pipeline con spaCy
nlp = spacy.load('es_core_news_sm')

def pipeline_spacy(texto):
    """Normalizado + lematizado"""
    # normalizar primero
    texto = texto.lower()
    texto = re.sub(r'[.,;:!?()\[\]{}"\']', '', texto)
    
    # procesar con spaCy
    doc = nlp(texto)
    lemmas = [token.lemma_ for token in doc if not token.is_space]
    return lemmas

print("Pipeline spaCy listo")

Pipeline spaCy listo
